In [4]:
# -*- coding: utf-8 -*-
import QUANTAXIS as QA
import pandas as pd
import numpy as np
import datetime

In [1]:
def MACD_JCSC(dataframe, SHORT=12, LONG=26, M=9):
    """
    1.DIF向上突破DEA，买入信号参考。
    2.DIF向下跌破DEA，卖出信号参考。
    """
    CLOSE = dataframe.close
    DIFF = QA.EMA(CLOSE, SHORT) - QA.EMA(CLOSE, LONG)
    DEA = QA.EMA(DIFF, M)
    MACD = 2*(DIFF-DEA)

    CROSS_JC = QA.CROSS(DIFF, DEA)
    CROSS_SC = QA.CROSS(DEA, DIFF)
    ZERO = 0
    return pd.DataFrame({'DIFF': DIFF, 'DEA': DEA, 'MACD': MACD, 'CROSS_JC': CROSS_JC, 'CROSS_SC': CROSS_SC, 'ZERO': ZERO})

In [65]:
def QSDD(dataframe, SHORT=12, LONG=26, M=9):
    """
    1.line_mid向上突破line_long，买入信号参考。
    2.line_mid向下跌破line_long，卖出信号参考。
    """
    OPEN = dataframe.open
    HIGH = dataframe.high
    LOW = dataframe.low
    CLOSE = dataframe.close

    # QSDD策略
    # A = talib.MA(-100 * (talib.MAX(HIGH, 34) - CLOSE) / (talib.MAX(HIGH, 34) - talib.MIN(LOW, 34)), 19)
    # B = -100 * (talib.MAX(HIGH, 14) - CLOSE) / (talib.MAX(HIGH, 14) - talib.MIN(LOW, 14))
    # D = talib.EMA(-100 * (talib.MAX(HIGH, 34) - CLOSE) / (talib.MAX(HIGH, 34) - talib.MIN(LOW, 34)), 4)
    A = QA.MA(-100 * (QA.HHV(HIGH, 34) - CLOSE) /
              (QA.HHV(HIGH, 34) - QA.LLV(LOW, 34)), 19)
    B = -100 * (QA.HHV(HIGH, 14) - CLOSE) / \
        (QA.HHV(HIGH, 14) - QA.LLV(LOW, 14))
    D = QA.EMA(-100 * (QA.HHV(HIGH, 34) - CLOSE) /
               (QA.HHV(HIGH, 34) - QA.LLV(LOW, 34)), 4)

    line_long = A + 100
    line_short = B + 100
    line_mid = D + 100  # 信号线

    CROSS_JC = QA.CROSS(line_mid, line_long)
    CROSS_SC = QA.CROSS(line_long, line_mid)
    return pd.DataFrame({'line_mid': line_mid, 'line_long': line_long, 'CROSS_JC': CROSS_JC, 'CROSS_SC': CROSS_SC})

In [2]:
import QUANTAXIS as QA
import numpy as np
# import talib
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# 定义MACD函数
def TA_MACD(prices, fastperiod=12, slowperiod=26, signalperiod=9):
    '''
    参数设置:
        fastperiod = 12
        slowperiod = 26
        signalperiod = 9

    返回: macd - dif, signal - dea, hist * 2 - bar, delta
    '''
    macd, signal, hist = talib.MACD(prices, 
                                    fastperiod=fastperiod, 
                                    slowperiod=slowperiod, 
                                    signalperiod=signalperiod)
    delta = np.r_[np.nan, np.diff(hist * 2)]
    return np.c_[macd, signal, hist * 2, delta]


# 定义MA函数
def TA_MA(prices, timeperiod=5):
    '''
    参数设置:
        timeperiod = 5

    返回: ma
    '''
    ma = talib.MA(prices, timeperiod=timeperiod)
    return ma


# 定义RSI函数
def TA_RSI(prices, timeperiod=12):
    '''
    参数设置:
        timeperiod = 12

    返回: ma
    '''
    rsi = talib.RSI(prices, timeperiod=timeperiod)
    delta = np.r_[np.nan, np.diff(rsi)]
    return np.c_[rsi, delta]


# 定义RSI函数
def TA_BBANDS(prices, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0):
    '''
    参数设置:
        timeperiod = 5
        nbdevup = 2
        nbdevdn = 2

    返回: up, middle, low
    '''
    up, middle, low = talib.BBANDS(prices, timeperiod, nbdevup, nbdevdn, matype)
    ch = (up - low) / low
    delta = np.r_[np.nan, np.diff(ch)]

    ma30 = TA_MA(prices, timeperiod=30)
    boll_band_channel_padding = (ma30 - low) / low
    padding_delta = np.r_[np.nan, np.diff(boll_band_channel_padding)]
    ch_ma30 = talib.MA(ch, timeperiod=30)
    return np.c_[up, middle, low, ch, ch_ma30, delta, padding_delta]


def TA_KDJ(hight, low, close, fastk_period=9, slowk_matype=0, slowk_period=3, slowd_period=3):
    '''
    参数设置:
        fastk_period = 0
        lowk_matype = 0, 
        slowk_period = 3, 
        slowd_period = 3

    返回: K, D, J
    '''
    K, D = talib.STOCH(hight, low, close, fastk_period=fastk_period, slowk_matype=slowk_matype, slowk_period=slowk_period, slowd_period=slowd_period)
    J = 3 * K - 2 * D
    delta = np.r_[np.nan, np.diff(J)]
    return np.c_[K, D, J, delta]


def TA_CCI(high, low, close, timeperiod=14):
    """
    名称：平均趋向指数的趋向指数
    简介：使用ADXR指标，指标判断ADX趋势。
    """
    real = talib.CCI(high, low, close, timeperiod=14)
    delta = np.r_[np.nan, np.diff(real)]
    return np.c_[real, delta]
#获取全市场股票 list格式
code  = QA.QA_fetch_stock_list_adv().code.tolist()
codelist = QA.QA_fetch_stock_block_adv().get_block('沪深300').code
print(codelist[0:30])

# 获取全市场数据 QADataStruct格式
data1 =  QA.QA_fetch_stock_day_adv(code, '2019-10-01','2020-02-18')
data2 =  QA.QA_fetch_stock_day_adv(codelist, '2019-10-01','2020-02-18')

['000001', '000002', '000063', '000066', '000069', '000100', '000157', '000166', '000333', '000338', '000425', '000538', '000568', '000596', '000625', '000627', '000651', '000656', '000661', '000671', '000703', '000708', '000723', '000725', '000728', '000768', '000776', '000783', '000786', '000858']


In [4]:
# 写个指标 so easy
def ifup20(data):
    # QA内建指标计算 Python原生代码
    return (QA.MA(data.close, 5)-QA.MA(data.close, 20)).dropna() > 0

def ifup20_TA(data):
    # TA-lib计算
    return (TA_MA(data.close, 5)-TA_MA(data.close, 20)).dropna() > 0

# apply到 QADataStruct上

# 写个自定义指标 MAX_FACTOR QA内建指标计算 Python原生代码
def ifmaxfactor_greater(data):
    RSI = QA.QA_indicator_RSI(data)
    CCI = QA.QA_indicator_CCI(data)
    KDJ = QA.QA_indicator_KDJ(data)
    MAX_FACTOR = CCI['CCI'] + (RSI['RSI1'] - 50) * 4 + (KDJ['KDJ_J'] - 50) * 4
    MAX_FACTOR_delta = np.r_[np.nan, np.diff(MAX_FACTOR)]
    REGRESSION_BASELINE = (RSI['RSI1'] - 50) * 4
    return ((MAX_FACTOR+MAX_FACTOR_delta)-(REGRESSION_BASELINE-133)).dropna() > 0

# 写个自定义指标 MAX_FACTOR TA-lib计算
def ifmaxfactor_greater_TA(data):
    RSI = TA_RSI(data.close)
    CCI = TA_CCI(data.high, data.low, data.close)
    KDJ = TA_KDJ(data.high, data.low, data.close)    
    MAX_FACTOR = CCI[:,0] + (RSI[:,0] - 50) * 4 + (KDJ[:,2] - 50) * 4
    MAX_FACTOR_delta = np.r_[np.nan, np.diff(MAX_FACTOR)]
    REGRESSION_BASELINE = (RSI[:,0] - 50) * 4
    return pd.DataFrame(((MAX_FACTOR+MAX_FACTOR_delta)-(REGRESSION_BASELINE-133)), index=data.index).dropna() > 0

In [6]:
ind_up20 = data1.add_func(ifup20)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


In [53]:
ind_up20
# .loc['2020-02-18',:]

date        code  
2019-11-04  000001    True
2019-11-05  000001    True
2019-11-06  000001    True
2019-11-07  000001    True
2019-11-08  000001    True
                      ... 
2020-02-18  300812    True
2020-02-17  002971    True
2020-02-18  002971    True
            300813    True
            688178    True
Name: close, Length: 258590, dtype: bool

In [8]:
ind_up20.groupby(level=0).sum()

date
2019-11-04    1129
2019-11-05    1072
2019-11-06    1062
2019-11-07    1160
2019-11-08    1184
              ... 
2020-02-12     885
2020-02-13     962
2020-02-14    1029
2020-02-17    1134
2020-02-18    1387
Name: close, Length: 70, dtype: int64

In [5]:
%timeit ind1 = data1.add_func(ifup20)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(
d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(
d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(
d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(
d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version

3.51 s ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit ind1 = data1.add_func(ifup20_TA)

4.48 s ± 44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit ind2 = data2.add_func(ifmaxfactor_greater)

7.59 s ± 84.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit ind2 = data2.add_func(ifmaxfactor_greater_TA)

1.24 s ± 36.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
#!/usr/bin/env python3

import datetime
import os

import pandas as pd

import QUANTAXIS as QA

# code_list = QA.QA_fetch_stock_list().index.tolist()
code_list = ["000001", "600000"]
current_date = str(datetime.date.today())
current_daily_bar = pd.DataFrame()
# 前置数据时间点
start_date = QA.QA_util_get_pre_trade_date(current_date, n=100)
# 前置数据准备
daily_data = QA.QA_fetch_stock_day_adv(
    code=code_list, start=start_date, end=current_date
).to_qfq()

# 方便计算，数据转换为相应透视数据
closes = daily_data.pivot("close")
highs = daily_data.pivot("high")
lows = daily_data.pivot("low")


def get_short_strength(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 27
) -> pd.Series:
    """
    计算空方力量

    :param highs: high 透视数据
    :param closes: close 透视数据
    :param period: 周期
    """
    A = highs.apply(QA.HHV, N=period, axis=0)
    B = lows.apply(QA.LLV, N=period, axis=0)
    return 100 * (A.iloc[-1] - closes.ffill().iloc[-1]) / (A.iloc[-1] - B.iloc[-1])


def get_long_strength(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 27
) -> pd.Series:
    """
    计算多方力量
    """
    A = highs.apply(QA.HHV, N=period, axis=0)
    B = lows.apply(QA.LLV, N=period, axis=0)
    C = ((closes - B) / (A - B)).apply(QA.QAIndicator.base.SMA, N=5, M=1, axis=0) * 100
    return (3 * C - 2 * C.apply(QA.QAIndicator.base.SMA, N=3, M=1)).iloc[-1]


def get_RSV(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 30
) -> pd.DataFrame:
    """
    计算 RSV 值
    """
    return (
        100
        * (closes - lows.apply(QA.LLV, N=period, axis=0))
        / (highs.apply(QA.HHV, N=period, axis=0) - lows.apply(QA.LLV, N=period, axis=0))
    )


def get_KDJ(
    highs: pd.DataFrame,
    lows: pd.DataFrame,
    closes: pd.DataFrame,
    period: int = 30,
    M1: int = 3,
    M2: int = 3,
) -> dict:
    """
    获取 KDJ 值
    """
    RSV = get_RSV(highs, lows, closes, period)
    KDJ_K = RSV.apply(QA.QAIndicator.base.SMA, N=3, M=1, axis=0)
    KDJ_D = KDJ_K.apply(QA.QAIndicator.base.SMA, N=3, M=1, axis=0)
    KDJ_J = 3 * KDJ_K - 2 * KDJ_D
    return {"KDJ_K": KDJ_K, "KDJ_D": KDJ_D, "KDJ_J": KDJ_J}

In [11]:
cn_stk_list = QA.QA_fetch_stock_list()
st_stk_list = cn_stk_list[cn_stk_list['name'].str.contains('ST|退')]

In [15]:
cn_stk_code = cn_stk_list['code'].to_list()

In [86]:
cn_stk_data = QA.QA_fetch_stock_day_adv(cn_stk_code,'2020-09-01','2021-03-07').to_qfq()

In [87]:
def EMAs(data):
    C = data.close
    H = data.high
    L = data.low
    O = data.open
    ema5 = QA.EMA(C,5)
    ema10 = QA.EMA(C,10)
    ema15 = QA.EMA(C,15)
    ema20 = QA.EMA(C,20)
    ema30 = QA.EMA(C,30)
    ema40 = QA.EMA(C,40)
    ema60 = QA.EMA(C,60)
    ema80 = QA.EMA(C,80)
    ema100 = QA.EMA(C,100)
    ema120 = QA.EMA(C,120)
    return pd.DataFrame({'ema5':ema5,'ema10':ema10,'ema15':ema15,'ema20':ema20,'ema30':ema30,'ema40':ema40,
                         'ema60':ema60,'ema80':ema80,'ema100':ema100,'ema120':ema120})

In [88]:
# cn_stk_data.data
cn_stk_data.add_func(EMAs)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


ema5       ema10       ema15       ema20       ema30  \
date       code                                                                 
2020-09-01 000001         NaN         NaN         NaN         NaN         NaN   
           000002         NaN         NaN         NaN         NaN         NaN   
           000004         NaN         NaN         NaN         NaN         NaN   
           000005         NaN         NaN         NaN         NaN         NaN   
           000006         NaN         NaN         NaN         NaN         NaN   
...                       ...         ...         ...         ...         ...   
2021-03-05 688777   87.336845   86.921730   86.894934   87.351051   88.757725   
           688788  116.821193  118.647864  119.790011  121.173809  125.306472   
           688819   45.407720   45.682256   46.186021   46.652659   47.323312   
           688981   58.364989   57.758925   57.449875   57.346125   57.385541   
           689009   72.526120   75.989204   79.075705   81.104773   82.924649   

                        ema40       ema60       ema80     ema100     ema120  
date       code                                                              
2020-09-01 000001         NaN         NaN         NaN        NaN        NaN  
           000002         NaN         NaN         NaN        NaN        NaN  
           000004         NaN         NaN         NaN        NaN        NaN  
           000005         NaN         NaN         NaN        NaN        NaN  
           000006         NaN         NaN         NaN        NaN        NaN  
...                       ...         ...         ...        ...        ...  
2021-03-05 688777   90.107456   92.055270         NaN        NaN        NaN  
           688788  130.148734  138.482358  144.279015        NaN        NaN  
           688819         NaN         NaN         NaN        NaN        NaN  
           688981   57.517590   57.767190   57.935391  58.043984  58.116579  
           689009   83.284196   82.788446   82.086984        NaN        NaN  

[496641 rows x 10 columns]

In [ ]:
cn_stk_data.pct_change

In [93]:
cn_stk_data[:]['pct_change'] = cn_stk_data.apply(lambda x:x.pct_change)

TypeError: 'QA_DataStruct_Stock_day' object does not support item assignment

In [95]:
cn_stk_data.variance

open    high     low   close    volume        amount  adj
date       code                                                               
2020-09-01 000001   14.96   15.23   14.88   15.14  813642.0  1.228343e+09  1.0
           000002   27.18   27.23   26.93   27.22  524296.0  1.420468e+09  1.0
           000004   26.41   26.44   24.64   24.99   70698.0  1.791782e+08  1.0
           000005    2.95    2.95    2.92    2.93   55320.0  1.619750e+07  1.0
           000006    6.35    6.38    6.18    6.32  306055.0  1.920837e+08  1.0
...                   ...     ...     ...     ...       ...           ...  ...
2021-03-05 688777   85.87   88.00   84.67   84.72   17137.0  1.471473e+08  1.0
           688788  114.01  115.85  112.12  115.50    3100.0  3.538687e+07  1.0
           688819   45.10   46.06   45.10   45.61   40191.0  1.834420e+08  1.0
           688981   58.00   58.91   57.71   58.52  281584.0  1.642365e+09  1.0
           689009   69.01   72.33   69.01   71.22   15661.0  1.121090e+08  1.0

[496641 rows x 7 columns]

In [77]:
def ma5(data):
    # QA内建指标计算 Python原生代码
    return pd.DataFrame({'ema1':QA.EMA(data.close, 20),'ema2':QA.EMA(data.close, 40),'ema3':QA.EMA(data.close, 60),'ema4':QA.EMA(data.close, 80)})

In [78]:
cn_stk_data.add_func(ma5)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


ema1        ema2        ema3        ema4
date       code                                                  
2020-01-02 000001         NaN         NaN         NaN         NaN
           000002         NaN         NaN         NaN         NaN
           000004         NaN         NaN         NaN         NaN
           000005         NaN         NaN         NaN         NaN
           000006         NaN         NaN         NaN         NaN
...                       ...         ...         ...         ...
2021-03-05 688777   87.351051   90.107456   92.055270         NaN
           688788  121.173809  130.148734  138.482358  144.279015
           688819   46.652659         NaN         NaN         NaN
           688981   57.346197   57.547205   57.959044   58.392165
           689009   81.104773   83.284196   82.788446   82.086984

[1109864 rows x 4 columns]

In [63]:
ind_emas = cn_stk_data.add_funcx(EMAs)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


In [69]:
ind_emas = QA.QA_indicator_EMA(cn_stk_data,20)

In [70]:
ind_emas

EMA
date       code              
2020-01-02 000001         NaN
           000002         NaN
           000004         NaN
           000005         NaN
           000006         NaN
...                       ...
2021-03-05 688777  117.813582
           688788  117.593241
           688819  110.737694
           688981  105.764580
           689009  102.474620

[1109864 rows x 1 columns]

In [67]:
ind_QSDD = cn_stk_data.add_funcx(QSDD)

In [68]:
ind_QSDD

,line_mid,line_long,CROSS_JC,CROSS_SC
date,,,,
2020-01-02,NaN,NaN,0,0
2020-01-03,NaN,NaN,0,0
2020-01-06,NaN,NaN,0,0
2020-01-07,NaN,NaN,0,0
2020-01-08,NaN,NaN,0,0
...,...,...,...,...
2021-03-04,NaN,NaN,0,0
2021-03-05,NaN,NaN,0,0
2021-03-03,NaN,NaN,0,0


In [39]:
cn_stk_data.data

,,open,high,low,close,volume,amount
date,code,,,,,,
2020-01-02,000001,16.65,16.95,16.55,16.87,1530231.0,2.571196e+09
2020-01-03,000001,16.94,17.31,16.92,17.18,1116194.0,1.914495e+09
2020-01-06,000001,17.01,17.34,16.91,17.07,862083.0,1.477930e+09
2020-01-07,000001,17.13,17.28,16.95,17.15,728607.0,1.247047e+09
2020-01-08,000001,17.00,17.05,16.63,16.66,847824.0,1.423609e+09
...,...,...,...,...,...,...,...
2021-03-01,000001,21.54,21.68,21.18,21.45,1125386.0,2.408051e+09
2021-03-02,000001,21.62,22.15,21.26,21.65,1473425.0,3.196537e+09
2021-03-03,000001,21.58,23.08,21.46,23.01,1919635.0,4.320025e+09


# 对区间最高和区间最低划线

In [96]:
# https://github.com/QUANTAXIS/QUANTAXIS/issues/1451
# 通达信公式:DRAWLINE(H=HHV(H,125),H,L=LLV(L,20),HHV(H,20),1);

In [97]:
import QUANTAXIS as QA

data = QA.QA_fetch_stock_day_adv('000001', '2019-01-01', '2020-02-12')

In [98]:
def qyzb(data):
    return QA.pd.DataFrame({
        'HHV120': QA.HHV(data.high,120),
        'LLV20': QA.LLV(data.low, 20),
        'HHV20': QA.HHV(data.high,20),
    })

In [99]:
ind = data.add_func(qyzb)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


In [100]:
# 首先找到HHV 对应的日期
data[data.high==ind.iloc[-1]['HHV120']].date

DatetimeIndex(['2019-10-14'], dtype='datetime64[ns]', name='date', freq=None)

In [101]:
# 再找到LLV 对应的日期
data[data.low==ind.iloc[-1]['LLV20']].date

DatetimeIndex(['2020-02-03'], dtype='datetime64[ns]', name='date', freq=None)

In [102]:
# 再找到终点(LLV20) 那天对应的HHV20的值 REINDEX 教他做人
ind['HHV20'].reindex(data[data.low==ind.iloc[-1]['LLV20']].index)

date        code  
2020-02-03  000001    17.34
Name: HHV20, dtype: float64

In [124]:
data =QA.QA_fetch_stock_day_adv(['000001','000002'], '2016-08-01','2019-09-01')

In [114]:
data =QA.QA_fetch_stock_day_adv('000001', '2016-08-01','2019-09-01')

# 多日涨幅

In [122]:
https://github.com/QUANTAXIS/QUANTAXIS/issues/1241
import pandas as pd
def drzf(data): #多日涨幅
    
    temp = {
        'open':data.open,
        'high':data.high,
        'low': data.low,
        'close': data.close,
        'volume': data.volume
    }
    y = {'close{}'.format(i): data.close.pct_change(i) for i in range(10)}
    temp.update(y)
    return pd.DataFrame(temp)

In [125]:
data.add_func(drzf)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


open   high    low  close     volume  close0    close1  \
date       code                                                              
2016-08-01 000001   9.18   9.34   9.17   9.28   759324.0     0.0       NaN   
           000002  17.01  17.29  16.90  17.17  1273062.0     0.0       NaN   
2016-08-02 000001   9.24   9.26   9.17   9.25   449165.0     0.0 -0.003233   
           000002  17.11  17.70  17.07  17.61  1604209.0     0.0  0.025626   
2016-08-03 000001   9.21   9.22   9.15   9.18   424622.0     0.0 -0.007568   
...                  ...    ...    ...    ...        ...     ...       ...   
2019-08-28 000002  26.55  26.69  26.36  26.55   266293.0     0.0  0.001131   
2019-08-29 000001  14.22  14.24  14.08  14.13   609034.0     0.0 -0.009811   
           000002  26.59  26.64  25.86  25.97   562064.0     0.0 -0.021846   
2019-08-30 000001  14.29  14.39  14.10  14.16   798500.0     0.0  0.002123   
           000002  26.22  26.22  25.68  25.80   434934.0     0.0 -0.006546   

                     close2    close3    close4    close5    close6    close7  \
date       code                                                                 
2016-08-01 000001       NaN       NaN       NaN       NaN       NaN       NaN   
           000002       NaN       NaN       NaN       NaN       NaN       NaN   
2016-08-02 000001       NaN       NaN       NaN       NaN       NaN       NaN   
           000002       NaN       NaN       NaN       NaN       NaN       NaN   
2016-08-03 000001 -0.010776       NaN       NaN       NaN       NaN       NaN   
...                     ...       ...       ...       ...       ...       ...   
2019-08-28 000002  0.003781 -0.015208 -0.022459 -0.030668 -0.022819 -0.012277   
2019-08-29 000001 -0.012579 -0.008421 -0.035495 -0.012579 -0.022145 -0.057372   
           000002 -0.020739 -0.018147 -0.036721 -0.043814 -0.051844 -0.044166   
2019-08-30 000001 -0.007708 -0.010482 -0.006316 -0.033447 -0.010482 -0.020069   
           000002 -0.028249 -0.027149 -0.024575 -0.043027 -0.050074 -0.058050   

                     close8    close9  
date       code                        
2016-08-01 000001       NaN       NaN  
           000002       NaN       NaN  
2016-08-02 000001       NaN       NaN  
           000002       NaN       NaN  
2016-08-03 000001       NaN       NaN  
...                     ...       ...  
2019-08-28 000002  0.010274  0.007973  
2019-08-29 000001 -0.052949 -0.051678  
           000002 -0.033854 -0.011796  
2019-08-30 000001 -0.055370 -0.050938  
           000002 -0.050423 -0.040179  

[1501 rows x 15 columns]

# 量能指标

In [126]:
def ind1(data,n1=5,n2=20):
    return pd.DataFrame({'res':QA.MA(data.volume,n1)>QA.MA(data.volume,n2)})

In [127]:
ind2 = data.add_func(ind1)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


In [129]:
ind2[ind2.res==True].tail(10)

res
date       code        
2019-08-22 000001  True
           000002  True
2019-08-23 000001  True
           000002  True
2019-08-26 000001  True
           000002  True
2019-08-27 000001  True
           000002  True
2019-08-29 000002  True
2019-08-30 000002  True

# 加入量能指标

In [131]:
import QUANTAXIS as QA
import pandas as pd
data = QA.QA_fetch_stock_day_adv('000001', '2019-01-01', '2020-02-12')

In [134]:
def mavol(data):
    temp = {
        'open' : data.open,
        'high' : data.high,
        'low' : data.low,
        'close' : data.close,
        'volume': data.volume
    }
    y = {'vol{}'.format(i): QA.MA(data.volume,i) for i in [10,60,240]}
    temp.update(y)
    return pd.DataFrame(temp)

In [135]:
data.add_func(mavol)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


,,open,high,low,close,volume,vol10,vol60,vol240
date,code,,,,,,,,
2019-01-02,000001,9.39,9.42,9.16,9.19,539386.0,NaN,NaN,NaN
2019-01-03,000001,9.18,9.33,9.15,9.28,415537.0,NaN,NaN,NaN
2019-01-04,000001,9.24,9.82,9.22,9.75,1481159.0,NaN,NaN,NaN
2019-01-07,000001,9.84,9.85,9.63,9.74,865687.0,NaN,NaN,NaN
2019-01-08,000001,9.73,9.74,9.62,9.66,402388.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-02-06,000001,14.81,14.87,14.51,14.77,1185815.0,1173883.5,828750.200000,1.042881e+06
2020-02-07,000001,14.60,14.69,14.41,14.62,924852.0,1163558.3,829775.650000,1.042735e+06
2020-02-10,000001,14.51,14.53,14.30,14.50,1339495.0,1236964.2,840235.633333,1.043577e+06


测试 QA indicator

In [9]:
# 获取所有股票列表
cn_stk_list = QA.QA_fetch_stock_list()
# 排除ST和退市股
cn_stk_list = cn_stk_list[~cn_stk_list['name'].str.contains('ST|退')]
# 获取20天交易日
today = datetime.date.today()
delta = datetime.timedelta(days=150)
start = (today - delta).strftime('%Y-%m-%d')
#     end = today.strftime('%Y-%m-%d')
end = '2021-03-15'
code_list = cn_stk_list.code.to_list()

In [26]:
# stk_day_all = QA.QA_fetch_stock_day_full_adv(code_list,start,end)
QA.QA_fetch_stock_day_adv(code_list,start,end)

QA Error QA_fetch_stock_day_adv parameter code=['000001', '000002', '000004', '000005', '000006', '000007', '000008', '000009', '000010', '000011', '000012', '000014', '000016', '000019', '000020', '000021', '000023', '000025', '000026', '000027', '000028', '000029', '000030', '000031', '000032', '000034', '000035', '000036', '000037', '000038', '000039', '000040', '000042', '000045', '000046', '000048', '000049', '000050', '000055', '000056', '000058', '000059', '000060', '000061', '000062', '000063', '000065', '000066', '000068', '000069', '000070', '000078', '000088', '000089', '000090', '000096', '000099', '000100', '000150', '000151', '000153', '000155', '000156', '000157', '000158', '000159', '000166', '000301', '000333', '000338', '000400', '000401', '000402', '000403', '000404', '000407', '000411', '000413', '000415', '000416', '000417', '000419', '000420', '000421', '000423', '000425', '000426', '000428', '000429', '000430', '000488', '000498', '000501', '000502', '000503', '0

In [19]:
stk_day_all.data()

TypeError: 'memoryview' object is not callable